# Решение для соревнования Avito Competition 2024

## Цель
Разработать предсказательную модель, которая максимально точно определит вероятность кликов пользователей на рекламные объявления на платформе Avito. Основные задачи включают анализ структуры данных, генерацию и отбор релевантных признаков, а также разработку нескольких моделей, чтобы улучшить точность на основе различных комбинаций признаков.

## Анализ данных и обоснование подхода

1. **Уникальные комбинации**:
   - **Общее покрытие**: Все `logcat_id` и `adv_campaign_id`, присутствующие в тренировочном наборе, также встречаются в тестовом.
   - **Пары и тройки**:
     - В тестовом наборе **90,520** уникальных пар `['user_id', 'logcat_id']` отсутствуют в тренировочном наборе.
     - Пары `['user_id', 'adv_campaign_id']` в тесте, которых нет в тренировочном наборе: **1,016,440**.
     - Общих комбинаций `['user_id', 'adv_campaign_id', 'logcat_id']` в обоих наборах данных — **966,847**.
   - **Пользователи**: В тесте есть **7** новых пользователей, отсутствующих в тренировочных данных.
   - **Размер данных**: Тестовый набор содержит **1,983,287** записей и **159,470** уникальных пользователей, а тренировочный — **114,741,035** записей и **3,263,615** пользователей.

2. **Вывод**:
   На основании анализа структуры данных предлагается использование трех отдельных моделей для различных комбинаций `user`, `logcat_id` и `adv_campaign_id`:
   - **Модель 1**: Для комбинации `user / logcat_id / adv_campaign_id`.
   - **Модель 2**: Для пары `user / logcat_id`.
   - **Модель 3**: Для всех остальных записей.

## Генерация признаков (фичей)

### Общие признаки для всех моделей

- **`user_ad_count`**: Общее количество показов рекламы для пользователя к моменту `event_date`.
- **`goal_budget_day`**: Среднесуточный бюджет рекламной кампании.
- **`day_of_week`**: День недели, отражающий временные паттерны взаимодействия с рекламой.
- **`click_rate`**: Кумулятивная кликабельность пользователя до текущей даты.
- **Кликабельность `adv_campaign_id` и `logcat_id`**: Определяет общую "кликабельность" для кампании и категории объявления по всем данным.
- **`days_since_campaign_start`**: Количество дней с начала кампании для конкретного пользователя, показывающее, насколько "свежа" реклама для пользователя.

### Особенности моделей и специфичные признаки

#### Модель 1: `user / logcat_id / adv_campaign_id`

- **`cumulative_shows_user_campaign_logcat`**: Кумулятивное количество показов для комбинации `user_id`, `logcat_id` и `adv_campaign_id`.
- **`daily_ad_count`**: Частота показов для конкретной комбинации кампании и категории в течение дня.
- **`days_since_last_interaction_adv_logcat`**: Время с момента последнего взаимодействия с конкретной кампанией.
- **`user_ad_count`**: Общее количество показов для пользователя.
- **`click_rate`**: Кумулятивная кликабельность пользователя.
- **Кликабельность `adv_campaign_id` и `logcat_id`**: Общая "кликабельность" для кампании и категории.
- **`days_since_campaign_start`**: Количество дней с начала рекламной кампании для пользователя.

#### Модель 2: `user / logcat_id`

- **`cumulative_shows_logcat`**: Кумулятивное количество показов `logcat_id` для конкретного пользователя.
- **`daily_logcat_count`**: Частота показов категории `logcat_id` в течение дня.
- **`days_since_last_interaction_logcat`**: Время с момента последнего взаимодействия с данной категорией.
- **`user_ad_count`**: Общее количество показов для пользователя.
- **`click_rate`**: Кумулятивная кликабельность пользователя.
- **Ранги пользователей по кликам**: Ранговые значения для `adv_campaign_id` и `logcat_id`, помогающие определить активных пользователей.
- **Кликабельность `adv_campaign_id` и `logcat_id`**: Общая "кликабельность" по кампании и категории.
- **`days_since_campaign_start`**: Дни с начала кампании для пользователя.

#### Модель 3: Для остальных комбинаций

Эта модель используется для оставшихся записей, которые не попадают в предыдущие модели. Она включает общий набор фичей, таких как `user_ad_count`, `goal_budget_day`, `click_rate`, и использует более общие признаки по категориям и кампаниям.

## Предсказание и объединение результатов

1. **Разделение данных**: Тестовый набор делится на три блока — `test_block_1`, `test_block_2`, `test_block_3` — для предсказаний, соответствующих каждой модели.
2. **Модели и предсказания**:
   - Модель 1 обучается на `train_block_1` и предсказывает вероятности на `test_block_1`.
   - Модель 2 обучается на `train_block_2` и предсказывает вероятности на `test_block_2`.
   - Модель 3 обучается на `train_block_3` и предсказывает вероятности на `test_block_3`.
3. **Объединение предсказаний**: Предсказания из трех блоков объединяются в один финальный результат. В итоговой таблице сохраняются только три столбца: `user_id`, `adv_campaign_id` и `predict`.

4. **Сохранение результатов**: Полученный результат сохраняется в файл `submission.csv` для отправки.

Такой подход позволяет гибко подойти к решению задачи, учитывая различия между пользователями, кампаниями и категориями объявлений, и минимизировать ошибки за счет индивидуального подхода для каждой из трех моделей.

In [ ]:
import pandas as pd
import dask.dataframe as dd
import seaborn as sns
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:,.2f}'.format
import numpy as np
import os
####

In [ ]:
TRAIN_DATA = 'Data/train.parquet'
TEST_DATA = 'Data/test.parquet.parquet'
COMPAIGNS_DATA = 'Data/campaigns_meta.parquet.parquet'
CATEGORIES_DATA = 'Data/categories.parquet.csv.parquet'
TARGET_NAME = 'target'
COL2DROP = ['user_id', 'event_date','start_date', 'end_date']


In [ ]:
# Загрузка и подготовка тренировочного набора данных
train_df = pd.read_parquet(TRAIN_DATA)
train_df['event_date'] = pd.to_datetime(train_df['event_date'])  # Преобразование столбца 'event_date' в формат даты
train_df['day_of_week'] = train_df['event_date'].dt.dayofweek  # Добавление столбца 'day_of_week'
train_df['is_main'] = train_df['is_main'].astype(int)  # Преобразование флага 'is_main' к типу int
train_df.drop(columns='platform_id', inplace=True)  # Удаление ненужного столбца 'platform_id'

# Загрузка метаданных о кампаниях и категориях
campaigns_meta = pd.read_parquet(COMPAIGNS_DATA)
categories_df = pd.read_parquet(CATEGORIES_DATA)

# Преобразование дат в 'campaigns_meta' и вычисление дополнительных признаков
campaigns_meta['start_date'] = pd.to_datetime(campaigns_meta['start_date'])
campaigns_meta['end_date'] = pd.to_datetime(campaigns_meta['end_date'])
campaigns_meta['duration_days'] = (campaigns_meta['end_date'] - campaigns_meta['start_date']).dt.days  # Продолжительность кампании в днях
campaigns_meta['goal_budget_day'] = campaigns_meta['goal_budget'] / campaigns_meta['duration_days']  # Бюджет кампании на день

# Удаление ненужных столбцов в 'campaigns_meta'
campaigns_meta = campaigns_meta.drop(columns=['end_date', 'goal_budget', 'duration_days', 'location_id'])

# Загрузка и подготовка тестового набора данных
test_df = pd.read_parquet(TEST_DATA)
test_df['event_date'] = pd.to_datetime(test_df['event_date'])  # Преобразование столбца 'event_date' в формат даты
test_df['day_of_week'] = test_df['event_date'].dt.dayofweek  # Добавление столбца 'day_of_week'
test_df['is_main'] = train_df['is_main'].astype(int)  # Преобразование флага 'is_main' к типу int

In [ ]:
train_df = pd.read_parquet(TRAIN_DATA)
# Предположим, что ваш целевой столбец называется 'target'
count_ones = train_df['target'].sum()
total_rows = len(train_df)

print(f"Количество единиц (1): {count_ones}")
print(f"Общее количество записей: {total_rows}")
print(f"Процент единиц: {count_ones / total_rows * 100:.2f}%")

## Разбивает Тест на три блока

In [ ]:
# Шаг 1: Добавление logcat_id
train_df = train_df.merge(
    campaigns_meta,
    on='adv_campaign_id',
    how='left',
    copy=False
)

test_df = test_df.merge(
    campaigns_meta,
    on='adv_campaign_id',
    how='left',
    copy=False
)

# Шаг 2: Создание блоков

# Устанавливаем индекс для train_df
train_df.set_index(['user_id', 'logcat_id', 'adv_campaign_id'], inplace=True)

# Копируем индексы для последующего использования
train_full_index = train_df.index

# Устанавливаем индекс для test_df
test_df.set_index(['user_id', 'logcat_id', 'adv_campaign_id'], inplace=True)

# Блок 1: Полное совпадение
block_1_index = test_df.index.intersection(train_full_index)
block_1 = test_df.loc[block_1_index]

# Удаляем блок 1 из test_df для дальнейшей обработки
test_df_remaining = test_df.drop(block_1_index)

# Сбрасываем индекс для установки нового
test_df_remaining = test_df_remaining.reset_index()

# Создаем индексы по (user_id, logcat_id)
train_user_logcat_index = train_df.reset_index().set_index(['user_id', 'logcat_id']).index.unique()
test_user_logcat_index = test_df_remaining.set_index(['user_id', 'logcat_id']).index

# Блок 2: Совпадение по user_id и logcat_id
block_2_index = test_user_logcat_index.intersection(train_user_logcat_index)
block_2 = test_df_remaining.set_index(['user_id', 'logcat_id']).loc[block_2_index]

# Блок 3: Оставшиеся записи
block_3 = test_df_remaining.set_index(['user_id', 'logcat_id']).drop(block_2_index)

# Шаг 3: Проверка целостности
total_rows = len(block_1) + len(block_2) + len(block_3)
assert total_rows == len(test_df), "Ошибка: блоки не суммируются в полный test_df"

# Сбрасываем индекс, если необходимо
block_1 = block_1.reset_index()
block_2 = block_2.reset_index()
block_3 = block_3.reset_index()

In [ ]:
# Загрузка и первичная обработка тренировочного набора данных
train_df = pd.read_parquet(TRAIN_DATA)

# Преобразование столбца 'event_date' к типу даты и добавление признака дня недели
train_df['event_date'] = pd.to_datetime(train_df['event_date'])
train_df['day_of_week'] = train_df['event_date'].dt.dayofweek

# Приведение флага 'is_main' к целому типу и удаление ненужного столбца 'platform_id'
train_df['is_main'] = train_df['is_main'].astype(int)
train_df.drop(columns='platform_id', inplace=True)

# Объединение с метаданными кампаний
train_df = train_df.merge(campaigns_meta, how='left', on='adv_campaign_id')

# Проверка на наличие пропусков в данных
missing_values = train_df.isna().sum()
print("Количество пропусков по каждому столбцу:\n", missing_values)

### Фичи общие для Модели3 (block_3)
- **user_ad_count**: Общее количество показов рекламы для пользователя на момент `event_date`.
- **click_rate**: Кумулятивная кликабельность пользователя.
- **Кликабельность для `adv_campaign_id` и `logcat_id`**: Определяет "кликабельность" для каждого `adv_campaign_id` и `logcat_id` по всему набору данных.
- **Время с начала кампании для пользователя**: Количество дней с первого показа кампании для пользователя, отражающее актуальность рекламы для пользователя.

In [ ]:
# Функция подсчета количества показов рекламы для каждого пользователя на момент event_date
train_df = train_df.sort_values(['user_id', 'event_date'])  # Сортировка по user_id и дате события
train_df['user_ad_count'] = train_df.groupby('user_id').cumcount()  # Кумулятивный счетчик показов рекламы для user_id

# Расчет кумулятивной кликабельности (click_rate) для каждого пользователя
train_df['cumulative_clicks'] = train_df.groupby('user_id')['target'].cumsum().shift(1).fillna(0)  # Сумма кликов до текущей даты
train_df['cumulative_shows'] = train_df.groupby('user_id').cumcount()  # Количество показов до текущей даты
train_df['click_rate'] = np.where(
    train_df['cumulative_shows'] > 0,
    train_df['cumulative_clicks'] / train_df['cumulative_shows'],
    0
)  # Кликабельность пользователя до текущего события

# Кумулятивный подсчет кликов и показов по logcat_id для расчета logcat_click_rate
train_df = train_df.sort_values(['logcat_id', 'event_date'])  # Сортировка по logcat_id и дате события
train_df['cumulative_clicks_logcat'] = train_df.groupby('logcat_id')['target'].cumsum().shift(1).fillna(0)  # Кумулятивные клики
train_df['cumulative_shows_logcat'] = train_df.groupby('logcat_id').cumcount()  # Кумулятивные показы
train_df['logcat_click_rate'] = np.where(
    train_df['cumulative_shows_logcat'] > 0,
    train_df['cumulative_clicks_logcat'] / train_df['cumulative_shows_logcat'],
    0
)  # Кликабельность для logcat_id до текущего события

# Подсчет дней с начала кампании для каждого пользователя и кампании
train_df = train_df.sort_values(['user_id', 'adv_campaign_id', 'event_date'])  # Сортировка по user_id, adv_campaign_id и дате события
train_df['days_since_campaign_start'] = (train_df['event_date'] - train_df['start_date']).dt.days  # Дней с начала кампании

In [ ]:
# Создаем DataFrame `train_block_3`, оставляя только нужные столбцы
train_block_3 = train_df[[
    'user_id',               # Уникальный идентификатор пользователя
    'adv_campaign_id',       # Идентификатор рекламной кампании
    'adv_creative_id',       # Идентификатор рекламного креатива
    'event_date',            # Дата события (показа рекламы или взаимодействия)
    'banner_code',           # Код баннера
    'is_main',               # Флаг основного показа для пользователя (1 - основной, 0 - нет)
    'target',                # Целевая переменная (взаимодействие пользователя с рекламой)
    'day_of_week',           # День недели, в который произошло событие
    'goal_cost',             # Стоимость достижения цели в рамках рекламной кампании
    'logcat_id',             # Идентификатор логической категории
    'goal_budget_day',       # Бюджет рекламной кампании в расчете на день
    'user_ad_count',         # Общее количество показов рекламы для пользователя на момент события
    'click_rate',            # Кумулятивная кликабельность пользователя до текущей даты
    'logcat_click_rate',     # Кумулятивная кликабельность для логической категории до текущей даты
    'days_since_campaign_start' # Количество дней с начала рекламной кампании для пользователя
]]

### Подготовим фичи block3 под Model3

In [ ]:
# Переименование block_3 на test_3 и сохранение оригинального количества строк
test_3 = block_3.copy()
original_length = len(test_3)  # Исходное количество строк для проверки целостности

# Шаг 1: Вычисляем `days_since_campaign_start` как разницу между `event_date` и `start_date`
test_3['days_since_campaign_start'] = (test_3['event_date'] - test_3['start_date']).dt.days
print("Столбцы после добавления days_since_campaign_start:", test_3.columns)

# Шаг 2: Добавляем последнее значение `user_ad_count` для каждого `user_id` из `train_block_3`
last_user_ad_count = train_block_3.groupby('user_id')['user_ad_count'].last().reset_index()
test_3 = test_3.merge(last_user_ad_count, on='user_id', how='left')
print("Столбцы после добавления user_ad_count:", test_3.columns)

# Шаг 3: Добавляем последнее значение `click_rate` для каждого `user_id` из `train_block_3`
last_click_rate = train_block_3.groupby('user_id')['click_rate'].last().reset_index()
test_3 = test_3.merge(last_click_rate, on='user_id', how='left')
print("Столбцы после добавления click_rate:", test_3.columns)

# Шаг 4: Добавляем последнее значение `logcat_click_rate` для каждого `logcat_id` из `train_block_3`
last_logcat_click_rate = train_block_3.groupby('logcat_id')['logcat_click_rate'].last().reset_index()
test_3 = test_3.merge(last_logcat_click_rate, on='logcat_id', how='left')
print("Столбцы после добавления logcat_click_rate:", test_3.columns)

# Проверка: убеждаемся, что количество строк не изменилось после всех операций
assert len(test_3) == original_length, "Ошибка: Количество строк изменилось после преобразований!"

### Фичи для Модели 1: `user / logcat_id / adv_campaign_id`

- **cumulative_shows_logcat**: Кумулятивное количество показов для комбинации `['user_id', 'adv_campaign_id', 'logcat_id']` до текущей `event_date`.
- **daily_ad_count**: Частота показов конкретной рекламной кампании и категории для пользователя в течение дня.
- **days_since_last_interaction_adv_logcat**: Время между взаимодействиями для каждого пользователя.
- **user_ad_count**: Общее количество показов для пользователя.
- **click_rate**: Кумулятивная кликабельность пользователя.
- **Кликабельность `adv_campaign_id` и `logcat_id`**: Определяет "кликабельность" для конкретного `adv_campaign_id` и `logcat_id` по всему набору данных.
- **Время с начала кампании**: Дни с момента первого показа кампании для пользователя.

In [ ]:
# Шаг 1: Загрузка тренировочного набора данных
train_df = pd.read_parquet(TRAIN_DATA)

# Шаг 2: Преобразование даты события и добавление дня недели
train_df['event_date'] = pd.to_datetime(train_df['event_date'])  # Преобразование даты к формату datetime
train_df['day_of_week'] = train_df['event_date'].dt.dayofweek  # Добавление признака "день недели"

# Шаг 3: Приведение флага 'is_main' к типу int для корректного использования
train_df['is_main'] = train_df['is_main'].astype(int)

# Шаг 4: Удаление ненужного столбца 'platform_id'
train_df.drop(columns='platform_id', inplace=True)

# Шаг 5: Присоединение метаданных кампании из campaigns_meta по 'adv_campaign_id'
train_df = train_df.merge(campaigns_meta, how='left', on='adv_campaign_id')

# Шаг 6: Проверка на наличие пропусков в данных после объединения
missing_values = train_df.isna().sum()
print("Количество пропусков в столбцах train_df после объединения с метаданными:")
print(missing_values)

In [ ]:
# Шаг 1: Предварительная сортировка данных
# Сортируем по 'user_id', 'logcat_id', 'adv_campaign_id' и 'event_date' для корректного выполнения кумулятивных расчетов
train_df = train_df.sort_values(['user_id', 'logcat_id', 'adv_campaign_id', 'event_date'])

# Шаг 2: Генерация признаков
# 2.1: Общее количество показов рекламы для каждого пользователя до текущей даты
train_df['user_ad_count'] = train_df.groupby('user_id').cumcount()

# 2.2: Общее количество показов для комбинации (user_id, adv_campaign_id, logcat_id) до текущей даты
train_df['cumulative_shows_user_campaign_logcat'] = train_df.groupby(['user_id', 'adv_campaign_id', 'logcat_id']).cumcount()

# 2.3: Кумулятивная кликабельность для комбинации (user_id, logcat_id, adv_campaign_id)
# Сначала считаем накопительные клики и показы, затем рассчитываем click_rate
train_df['cumulative_clicks_adv_logcat'] = train_df.groupby(['user_id', 'logcat_id', 'adv_campaign_id'])['target'].cumsum().shift(1).fillna(0)
train_df['cumulative_shows_adv_logcat'] = train_df.groupby(['user_id', 'logcat_id', 'adv_campaign_id']).cumcount()
train_df['click_rate_adv_logcat'] = np.where(
    train_df['cumulative_shows_adv_logcat'] > 0,
    train_df['cumulative_clicks_adv_logcat'] / train_df['cumulative_shows_adv_logcat'],
    0
)

# 2.4: Кумулятивная кликабельность для комбинации (logcat_id, adv_campaign_id) по всему датасету
# Сначала считаем накопительные клики и показы, затем рассчитываем logcat_campaign_click_rate
train_df['cumulative_clicks_logcat_campaign'] = train_df.groupby(['logcat_id', 'adv_campaign_id'])['target'].cumsum().shift(1).fillna(0)
train_df['cumulative_shows_logcat_campaign'] = train_df.groupby(['logcat_id', 'adv_campaign_id']).cumcount()
train_df['logcat_campaign_click_rate'] = np.where(
    train_df['cumulative_shows_logcat_campaign'] > 0,
    train_df['cumulative_clicks_logcat_campaign'] / train_df['cumulative_shows_logcat_campaign'],
    0
)

# 2.5: Количество дней с начала кампании для каждого пользователя
train_df['days_since_campaign_start'] = (train_df['event_date'] - train_df['start_date']).dt.days

# 2.6: Частота показов (количество показов в течение дня) для комбинации (user_id, logcat_id, adv_campaign_id)
train_df['daily_ad_count'] = train_df.groupby(['user_id', 'logcat_id', 'adv_campaign_id', 'event_date'])['event_date'].transform('count')

# 2.7: Время с последнего взаимодействия для комбинации (user_id, adv_campaign_id, logcat_id)
train_df['days_since_last_interaction_adv_logcat'] = train_df.groupby(['user_id', 'adv_campaign_id', 'logcat_id'])['event_date'].diff().dt.days.fillna(0)

# Печать структуры данных после добавления всех признаков
print(train_df.info())

In [ ]:
# Создаем DataFrame train_block_1 с выбранными признаками из train_df
train_block_1 = train_df[
    [
        'user_id',                  # Идентификатор пользователя
        'adv_campaign_id',          # Идентификатор рекламной кампании
        'adv_creative_id',          # Идентификатор рекламного креатива
        'event_date',               # Дата события
        'banner_code',              # Код баннера
        'is_main',                  # Признак основного баннера
        'target',                   # Целевая переменная
        'day_of_week',              # День недели
        'goal_cost',                # Стоимость цели
        'logcat_id',                # Идентификатор категории
        'goal_budget_day',          # Бюджет на день
        'user_ad_count',            # Общее количество показов рекламы для пользователя
        'cumulative_shows_user_campaign_logcat',  # Кумулятивные показы для комбинации user/campaign/logcat
        'click_rate_adv_logcat',    # Кумулятивная кликабельность для user/campaign/logcat
        'logcat_campaign_click_rate', # Кумулятивная кликабельность по logcat_id и campaign_id
        'days_since_campaign_start', # Время с начала кампании для пользователя
        'daily_ad_count',           # Частота показов в течение дня для комбинации user/campaign/logcat
        'days_since_last_interaction_adv_logcat' # Время с последнего взаимодействия
    ]
]

In [ ]:
# Копируем block_1 в test_1 для дальнейших преобразований
test_1 = block_1.copy()
print("Изначальные колонки:", test_1.columns)

# Сохраняем исходное количество строк в test_1 для проверки после преобразований
original_length = len(test_1)

# Шаг 1: Вычисляем days_since_campaign_start как разницу между event_date и start_date
test_1['days_since_campaign_start'] = (test_1['event_date'] - test_1['start_date']).dt.days
print("Добавлена days_since_campaign_start:", test_1.columns)

# Шаг 2: Присоединяем последнее значение user_ad_count для каждого user_id из train_block_1
last_user_ad_count = train_block_1.groupby('user_id')['user_ad_count'].last().reset_index()
test_1 = test_1.merge(last_user_ad_count, on='user_id', how='left')
print("Добавлена user_ad_count:", test_1.columns)

# Шаг 3: Присоединяем последнее значение cumulative_shows_user_campaign_logcat для каждой комбинации
last_cumulative_shows = train_block_1.groupby(['user_id', 'adv_campaign_id', 'logcat_id'])['cumulative_shows_user_campaign_logcat'].last().reset_index()
test_1 = test_1.merge(last_cumulative_shows, on=['user_id', 'adv_campaign_id', 'logcat_id'], how='left')
print("Добавлена cumulative_shows_user_campaign_logcat:", test_1.columns)

# Шаг 4: Присоединяем последнее значение click_rate_adv_logcat для комбинаций user_id, logcat_id и adv_campaign_id
last_click_rate_adv_logcat = train_block_1.groupby(['user_id', 'logcat_id', 'adv_campaign_id'])['click_rate_adv_logcat'].last().reset_index()
test_1 = test_1.merge(last_click_rate_adv_logcat, on=['user_id', 'logcat_id', 'adv_campaign_id'], how='left')
print("Добавлена click_rate_adv_logcat:", test_1.columns)

# Шаг 5: Присоединяем последнее значение logcat_campaign_click_rate для комбинаций logcat_id и adv_campaign_id
last_logcat_campaign_click_rate = train_block_1.groupby(['logcat_id', 'adv_campaign_id'])['logcat_campaign_click_rate'].last().reset_index()
test_1 = test_1.merge(last_logcat_campaign_click_rate, on=['logcat_id', 'adv_campaign_id'], how='left')
print("Добавлена logcat_campaign_click_rate:", test_1.columns)

# Шаг 6: Вычисляем daily_ad_count - количество показов в течение дня по каждой комбинации user_id, logcat_id, adv_campaign_id
test_1['daily_ad_count'] = test_1.groupby(['user_id', 'logcat_id', 'adv_campaign_id', 'event_date'])['event_date'].transform('count')
print("Добавлена daily_ad_count:", test_1.columns)

# Шаг 7: Присоединяем последнее значение days_since_last_interaction_adv_logcat для каждой комбинации
last_days_since_interaction = train_block_1.groupby(['user_id', 'adv_campaign_id', 'logcat_id'])['days_since_last_interaction_adv_logcat'].last().reset_index()
test_1 = test_1.merge(last_days_since_interaction, on=['user_id', 'adv_campaign_id', 'logcat_id'], how='left')
print("Добавлена days_since_last_interaction_adv_logcat:", test_1.columns)

# Проверка: убедимся, что количество строк в test_1 не изменилось после всех преобразований
assert len(test_1) == original_length, "Ошибка: Количество строк изменилось после преобразований!"

### Фичи для Модели 2: `user / logcat_id`

- **cumulative_shows_logcat**: Кумулятивное количество показов `logcat_id` до текущей `event_date`.
- **daily_logcat_count**: Частота показов категории для пользователя в течение дня.
- **days_since_last_interaction_logcat**: Время между взаимодействиями для пользователя.
- **user_ad_count**: Общее количество показов для пользователя.
- **click_rate**: Кумулятивная кликабельность пользователя.
- **Ранги пользователей по кликам**: Ранги пользователей по числу кликов для `adv_campaign_id` и `logcat_id`, помогающие определить активность.
- **Кликабельность для `adv_campaign_id` и `logcat_id`**: Измеряет "кликабельность" для каждого `adv_campaign_id` и `logcat_id` по всему набору данных.
- **Время с начала кампании для пользователя**: Дни с момента первого показа кампании, влияющие на интерес к рекламе.

In [ ]:
# Загрузка и подготовка основного DataFrame
train_df = pd.read_parquet(TRAIN_DATA)

# Преобразование столбца `event_date` к типу datetime
train_df['event_date'] = pd.to_datetime(train_df['event_date'])

# Извлечение дня недели из `event_date` и добавление в новый столбец `day_of_week`
train_df['day_of_week'] = train_df['event_date'].dt.dayofweek

# Приведение столбца `is_main` к типу int для корректного хранения флага
train_df['is_main'] = train_df['is_main'].astype(int)

# Удаление ненужного столбца `platform_id` из DataFrame
train_df.drop(columns='platform_id', inplace=True)

# Объединение с метаданными рекламных кампаний (`campaigns_meta`) по ключу `adv_campaign_id`
train_df = train_df.merge(campaigns_meta, how='left', on='adv_campaign_id')

# Проверка на наличие пропущенных значений в столбцах DataFrame
missing_values = train_df.isna().sum()
print("Количество пропущенных значений в каждом столбце:")
print(missing_values)

In [ ]:
# Сортируем данные один раз в начале по user_id, logcat_id и event_date
train_df = train_df.sort_values(['user_id', 'logcat_id', 'event_date'])

# Шаг 1: Создаем фичу user_ad_count — количество показов рекламы для каждого пользователя до текущей даты (не включая текущую запись)
train_df['user_ad_count'] = train_df.groupby('user_id').cumcount()

# Шаг 2: Кумулятивное количество показов и кликов для комбинации user_id и logcat_id до текущей даты
train_df['cumulative_shows_logcat'] = train_df.groupby(['user_id', 'logcat_id']).cumcount()
train_df['cumulative_clicks_logcat'] = (
    train_df.groupby(['user_id', 'logcat_id'])['target']
    .cumsum()
    .shift(1)  # Смещение на одну строку для предотвращения утечки информации
    .fillna(0)
)

# Расчет кликабельности (click_rate_logcat) для каждого user_id и logcat_id
train_df['click_rate_logcat'] = np.where(
    train_df['cumulative_shows_logcat'] > 0,
    train_df['cumulative_clicks_logcat'] / train_df['cumulative_shows_logcat'],
    0
)

# Шаг 3: Кумулятивное количество показов и кликов для каждого logcat_id по всему датасету
train_df['cumulative_clicks_logcat_total'] = (
    train_df.groupby('logcat_id')['target']
    .cumsum()
    .shift(1)
    .fillna(0)
)
train_df['cumulative_shows_logcat_total'] = train_df.groupby('logcat_id').cumcount()

# Расчет общей кликабельности (logcat_click_rate) для logcat_id по всему датасету
train_df['logcat_click_rate'] = np.where(
    train_df['cumulative_shows_logcat_total'] > 0,
    train_df['cumulative_clicks_logcat_total'] / train_df['cumulative_shows_logcat_total'],
    0
)

# Шаг 4: Вычисление количества дней с начала кампании для каждого показа
train_df['days_since_campaign_start'] = (train_df['event_date'] - train_df['start_date']).dt.days

# Шаг 5: Подсчет ежедневного количества показов для user_id и logcat_id, исключая текущую запись
train_df['daily_ad_count_logcat'] = (
    train_df.groupby(['user_id', 'logcat_id', 'event_date'])['event_date']
    .transform('count')
    .shift(1)
    .fillna(0)
)

# Шаг 6: Расчет времени (в днях) с последнего взаимодействия для каждой комбинации user_id и logcat_id
train_df['days_since_last_interaction_logcat'] = (
    train_df.groupby(['user_id', 'logcat_id'])['event_date']
    .diff()
    .dt.days
    .fillna(0)
)

In [ ]:
# Создание DataFrame train_block_2 с отобранными столбцами и их пояснениями
train_block_2 = train_df[[
    'user_id',                 # ID пользователя
    'adv_campaign_id',         # ID рекламной кампании
    'adv_creative_id',         # ID рекламного креатива
    'event_date',              # Дата события
    'banner_code',             # Код баннера
    'is_main',                 # Флаг основного рекламного материала
    'target',                  # Целевая переменная (клик или не клик)
    'day_of_week',             # День недели для события
    'goal_cost',               # Стоимость цели для кампании
    'logcat_id',               # ID категории логов
    'goal_budget_day',         # Среднедневной бюджет для рекламной кампании
    'user_ad_count',           # Счетчик общего числа показов рекламы для пользователя до текущей даты
    'cumulative_shows_logcat', # Кумулятивное количество показов для user_id и logcat_id на момент event_date
    'click_rate_logcat',       # Кликовая активность пользователя по logcat_id (относительная кликабельность)
    'logcat_click_rate',       # Общая кликабельность logcat_id по всему датасету
    'days_since_campaign_start', # Время (в днях) с начала кампании на момент event_date
    'daily_ad_count_logcat',   # Количество показов logcat_id для пользователя в конкретный день
    'days_since_last_interaction_logcat' # Дни с последнего взаимодействия с logcat_id для user_id
]]

In [ ]:
# Переименование block_2 на test_2 и создание копии для работы
test_2 = block_2.copy()
print("Начальные столбцы test_2:", test_2.columns)

# Сохраняем исходное количество строк в test_2 для контроля целостности данных
original_length = len(test_2)

# 1. Добавление последнего значения `user_ad_count` для каждого пользователя
# Извлекаем последнее значение user_ad_count из train_block_2 для каждого user_id
last_user_ad_count = train_block_2.groupby('user_id')['user_ad_count'].last().reset_index()
# Объединяем с test_2, добавляя `user_ad_count` по user_id
test_2 = test_2.merge(last_user_ad_count, on='user_id', how='left')
print("Столбцы после добавления user_ad_count:", test_2.columns)

# 2. Добавление последнего значения cumulative_shows_logcat по каждому user_id и logcat_id
# Извлекаем последнее значение cumulative_shows_logcat из train_block_2
last_cumulative_shows_logcat = train_block_2.groupby(['user_id', 'logcat_id'])['cumulative_shows_logcat'].last().reset_index()
# Объединяем с test_2, добавляя cumulative_shows_logcat для каждого сочетания user_id и logcat_id
test_2 = test_2.merge(last_cumulative_shows_logcat, on=['user_id', 'logcat_id'], how='left')
print("Столбцы после добавления cumulative_shows_logcat:", test_2.columns)

# 3. Добавление последнего значения click_rate_logcat для каждого user_id и logcat_id
# Извлекаем последнее значение click_rate_logcat для каждого user_id и logcat_id из train_block_2
last_click_rate_logcat = train_block_2.groupby(['user_id', 'logcat_id'])['click_rate_logcat'].last().reset_index()
# Объединяем с test_2, добавляя click_rate_logcat
test_2 = test_2.merge(last_click_rate_logcat, on=['user_id', 'logcat_id'], how='left')
print("Столбцы после добавления click_rate_logcat:", test_2.columns)

# 4. Добавление последнего значения logcat_click_rate по logcat_id
# Извлекаем последнее значение logcat_click_rate по logcat_id
last_logcat_click_rate = train_block_2.groupby('logcat_id')['logcat_click_rate'].last().reset_index()
# Объединяем с test_2, добавляя logcat_click_rate для каждого logcat_id
test_2 = test_2.merge(last_logcat_click_rate, on='logcat_id', how='left')
print("Столбцы после добавления logcat_click_rate:", test_2.columns)

# 5. Вычисление days_since_campaign_start — количество дней с начала кампании
# Сортируем данные по user_id, adv_campaign_id и event_date для корректного вычисления
test_2 = test_2.sort_values(['user_id', 'adv_campaign_id', 'event_date'])
# Рассчитываем days_since_campaign_start как разницу между event_date и start_date
test_2['days_since_campaign_start'] = (test_2['event_date'] - test_2['start_date']).dt.days

# 6. Подсчет показов logcat_id для каждого пользователя в день (daily_ad_count_logcat)
# Группируем по user_id, logcat_id и event_date и считаем количество показов
test_2['daily_ad_count_logcat'] = test_2.groupby(['user_id', 'logcat_id', 'event_date'])['event_date'].transform('count')
print("Столбцы после добавления days_since_campaign_start и daily_ad_count_logcat:", test_2.columns)

# 7. Добавление последнего значения days_since_last_interaction_logcat для каждого user_id и logcat_id
# Извлекаем последнее значение days_since_last_interaction_logcat из train_block_2
last_days_since_interaction_logcat = train_block_2.groupby(['user_id', 'logcat_id'])['days_since_last_interaction_logcat'].last().reset_index()
# Объединяем с test_2, добавляя days_since_last_interaction_logcat
test_2 = test_2.merge(last_days_since_interaction_logcat, on=['user_id', 'logcat_id'], how='left')
print("Столбцы после добавления days_since_last_interaction_logcat:", test_2.columns)

# Проверка: контрольный assert для проверки, что количество строк осталось неизменным
assert len(test_2) == original_length, "Ошибка: Количество строк изменилось после преобразований!"

In [ ]:
# Получаем количество строк в каждом блоке для верификации
original_test_rows = len(test_df)
block_1_rows = len(test_1)
block_2_rows = len(test_2)
block_3_rows = len(test_3)

# Суммируем количество строк в каждом блоке и сравниваем с исходным количеством строк в test_df
total_block_rows = block_1_rows + block_2_rows + block_3_rows

# Проверка совпадения количества строк: должно быть равно исходному количеству строк test_df
assert original_test_rows == total_block_rows, (
    f"Ошибка: Количество строк не совпадает! "
    f"Оригинальный тестовый набор: {original_test_rows}, "
    f"Сумма строк в блоках: {total_block_rows}"
)

# Вывод успешного прохождения проверки
print("Проверка пройдена успешно: Количество строк совпадает.")

In [ ]:
import os

# Создаем папку для сохранения блоков данных, если она еще не существует
output_dir = "data_split"
os.makedirs(output_dir, exist_ok=True)

# Сохраняем тренировочные блоки в формате Parquet
train_block_1.to_parquet(os.path.join(output_dir, 'train_block_1.parquet'))
train_block_2.to_parquet(os.path.join(output_dir, 'train_block_2.parquet'))
train_block_3.to_parquet(os.path.join(output_dir, 'train_block_3.parquet'))

# Сохраняем тестовые блоки в формате Parquet
test_1.to_parquet(os.path.join(output_dir, 'test_block_1.parquet'))
test_2.to_parquet(os.path.join(output_dir, 'test_block_2.parquet'))
test_3.to_parquet(os.path.join(output_dir, 'test_block_3.parquet'))

# Вывод сообщения об успешном сохранении файлов
print("Все блоки успешно сохранены в папку 'data_split'.")